In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import PySpin

In [3]:
# Get system
system = PySpin.System.GetInstance()
# Get camera list
cam_list = system.GetCameras()

print(f"Number of cameras available: {len(cam_list)}")

SpinnakerException: Spinnaker: System instance cannot be acquired. [-1012]

In [ ]:
# use spinview to get serial numbers for each camera

In [ ]:
camera_list = [
    {'name': 'top', 'serial': 22181547},
    {'name': 'side1', 'serial': 22181612},
]

In [ ]:
from datetime import datetime

In [ ]:
import csv
from multicamera_acquisition.paths import ensure_dir
from multicamera_acquisition.acquisition import Writer, AcquisitionLoop
import serial
from pathlib2 import Path
from datetime import datetime
import multiprocessing as mp
import warnings

In [ ]:
def acquire_video(
    save_location,
    camera_list,
    recording_duration_s,
    framerate = 30,
    exposure_time = 2000,
    serial_timeout_duration_s=0.1,
    overwrite=False,
    append_datetime=True,
    verbose = True,
):

    # get Path of save location
    if type(save_location) != Path:
        save_location = Path(save_location)

    # create a subfolder for the current datetime
    if append_datetime:
        date_str = datetime.now().strftime("%y-%m-%d-%H-%M-%S-%f")
        save_location = save_location / date_str

    # ensure that a directory exists to save data in
    ensure_dir(save_location)

    triggerdata_file = save_location / "triggerdata.csv"
    if triggerdata_file.exists() and (overwrite == False):
        raise FileExistsError(f"CSV file {triggerdata_file} already exists")


    # initialize cameras
    writers = []
    acquisition_loops = []

    # create acquisition loops
    for camera_dict in camera_list:
        name = camera_dict["name"]
        serial_number = camera_dict["serial"]

        video_file = save_location / f"{name}.{serial_number}.avi"
        metadata_file = save_location / f"{name}.{serial_number}.triggerdata.csv"
        
        if video_file.exists() and (overwrite == False):
            raise FileExistsError(f"Video file {video_file} already exists")

        # create a writer queue
        write_queue = mp.Queue()
        writer = Writer(
            write_queue,
            video_file_name=video_file,
            metadata_file_name=metadata_file,
            fps=framerate,
            camera_serial=serial_number,
            camera_name=name,
        )

        # prepare the acuqisition loop in a separate thread
        acquisition_loop = AcquisitionLoop(
            write_queue, serial_number=serial_number, exposure_time=exposure_time, gain=15
        )

        # initialize acquisition
        writer.start()
        writers.append(writer)
        acquisition_loop.start()
        acquisition_loop.ready.wait()
        acquisition_loops.append(acquisition_loop)
        if verbose:
            warnings.info(f"Initialized {name} ({serial_number})")

    # prepare acquisition loops
    for acquisition_loop in acquisition_loops:
        acquisition_loop.prime()
        acquisition_loop.ready.wait()

    # prepare communication with arduino
    serial_ports = glob.glob("/dev/ttyACM*")
    # check that there is an arduino available
    if len(serial_ports) == 0:
        raise ValueError("No serial device (i.e. Arduino) available to capture frames")
    port = glob.glob("/dev/ttyACM*")[0]
    arduino = serial.Serial(port=port, timeout=serial_timeout_duration_s)

    # delay recording to allow serial connection to connect
    time.sleep(1.0)

    # create a triggerdata file
    with open(triggerdata_file, "w") as triggerdata_f:
        triggerdata_writer = csv.writer(triggerdata_f)
        triggerdata_writer.writerow(
            ["pulse_id", "arduino_ms"]
            + [f"flag_{i}" for i in range(n_input_trigger_states)]
        )

    # Tell the arduino to start recording by sending along the recording parameters
    msg = b"".join(map(packIntAsLong, params))
    arduino.write(msg)

    # Run acquision
    confirmation = wait_for_serial_confirmation("Start")
    ## TODO: recieve channels flipped
    for i in tqdm(range(int(recording_duration_s / timeout_duration_s))):
        confirmation = arduino.readline().decode("utf-8").strip("\r\n")

        # save input data flags
        if len(confirmation) > 0:
            print(confirmation)
            if confirmation[:7] == "input: ":
                with open(triggerdata_file, "a") as triggerdata_f:
                    triggerdata_writer = csv.writer(triggerdata_f)
                    states = confirmation[7:].split(",")[:-2]
                    frame_num = confirmation[7:].split(",")[-2]
                    arduino_clock = confirmation[7:].split(",")[-1]
                    triggerdata_writer.writerow([frame_num, arduino_clock] + states)
            if verbose:
                print(confirmation)

        if confirmation == "Finished":
            break

    if confirmation != "Finished":
        confirmation = wait_for_serial_confirmation("Finished")

    # end acquisition loops
    for acquisition_loop in acquisition_loops:
        acquisition_loop.stop()
        acquisition_loop.join()

    # @CALEB: what is the purpose of this?
    for writer in writers:
        writer.join()

    if verbose:
        # count each frame
        for camera_dict in camera_list:
            name = camera_dict["name"]
            serial_number = camera_dict["serial"]
            video_file = save_location / f"{name}.{serial_number}.avi"
            print(f"Frames ({name}):", count_frames(f"{prefix}.{k}.{sn}.avi"))

In [ ]:
from multicamera_acquisition.paths import DATA_DIR

In [ ]:
import multicamera_acquisition.acquisition

In [ ]:
save_location = DATA_DIR / 'tests'

In [ ]:
acquire_video(
    save_location,
    camera_list,
    framerate = 30,
    exposure_time = 2000,
    recording_duration_s = 10,
    serial_timeout_duration_s=0.1,
    overwrite=False,
    append_datetime=True,
)